In [1]:
import csv
import os
import re
import time
import plotly.plotly as py
import plotly
from plotly import tools
from plotly.offline import init_notebook_mode
from plotly.graph_objs import *
import plotly.tools as tls

plotly.tools.set_credentials_file(username='sehwchoi', api_key='Ze9cXbcfuewb3C8TLktc')
init_notebook_mode(connected=True)  

In [2]:
mention_path = "../../user_network_data/aggregated_mention_networks"
user_set_path = "../../user_network_data/Event Users"

In [3]:
def __read_user_set(file_name):
    user_set = []
    with open(file_name) as file:
        user_set_data = csv.reader(file)
        for row in user_set_data:
            user_id = row[0]
            if user_id != "id":
                user_set.append(int(user_id))
    return user_set


In [34]:
user_set_by_event = {}
def read_user_set_for_all_events(path):
    for root, dirs, files in os.walk(path):
        for filename in files:
            file_match = re.search("(\d+)_", filename)
            if file_match:
                event_id = file_match.group(1)
                user_set = __read_user_set(os.path.join(root, filename))
                # print("process event: {} user_set_len: {}".format(event_id, len(user_set)))
                user_set_by_event[int(event_id)] = user_set
    print("user_set length: {}".format(len(user_set_by_event)))

read_user_set_for_all_events(user_set_path)
    

process event: 100 user_set_len: 8906
process event: 101 user_set_len: 5875
process event: 102 user_set_len: 1691
process event: 103 user_set_len: 33237
process event: 104 user_set_len: 14354
process event: 105 user_set_len: 8063
process event: 106 user_set_len: 48681
process event: 107 user_set_len: 108166
process event: 108 user_set_len: 0
process event: 109 user_set_len: 16151
process event: 110 user_set_len: 0
process event: 111 user_set_len: 108351
process event: 112 user_set_len: 22719
process event: 113 user_set_len: 73479
process event: 114 user_set_len: 98113
process event: 115 user_set_len: 42859
process event: 116 user_set_len: 39020
process event: 118 user_set_len: 8742
process event: 119 user_set_len: 333098
process event: 120 user_set_len: 16211
process event: 121 user_set_len: 18800
process event: 122 user_set_len: 68
process event: 123 user_set_len: 3874
process event: 124 user_set_len: 418
process event: 125 user_set_len: 0
process event: 126 user_set_len: 0
process ev

process event: 329 user_set_len: 92826
process event: 330 user_set_len: 161
process event: 331 user_set_len: 124935
process event: 332 user_set_len: 264719
process event: 333 user_set_len: 0
process event: 334 user_set_len: 0
process event: 335 user_set_len: 3385
process event: 336 user_set_len: 3459
process event: 337 user_set_len: 0
process event: 338 user_set_len: 0
process event: 339 user_set_len: 8180
process event: 340 user_set_len: 16290
process event: 341 user_set_len: 4386
process event: 342 user_set_len: 1401
process event: 343 user_set_len: 2222
process event: 344 user_set_len: 0
process event: 345 user_set_len: 1820
process event: 346 user_set_len: 86
process event: 42 user_set_len: 691
process event: 43 user_set_len: 68
process event: 44 user_set_len: 2975
process event: 45 user_set_len: 7590
process event: 46 user_set_len: 9640
process event: 47 user_set_len: 12527
process event: 48 user_set_len: 21650
process event: 49 user_set_len: 26066
process event: 50 user_set_len: 

In [42]:
def count_unique_user2(file_name, event):
    usr_unique_mention_ct = {}
    user_set = user_set_by_event[event]

    with open(file_name) as file:
        data = csv.reader(file)
        for pair in data:
            try:
                user1 = int(pair[0])
                user2 = int(pair[1])
            except Exception as e:
                print("file: {} error: {}".format(file_name, e))
            # print("user1: {} user2: {}".format(user1, user2))
            # check if user1 exist as a key and increment mention count1
            if user1 not in usr_unique_mention_ct:
                # count for an unique user2
                count1 = 1
                count2 = 0
                usr_unique_mention_ct[user1] = [count1, count2]
            else:
                usr_unique_mention_ct[user1][0] +=1
    
            # increment count2 if the user2 is in the set
            if user2 in user_set:
                #print("user2: {} in user_set".format(user2))
                usr_unique_mention_ct[user1][1] +=1
            
            #print("user1_mention: {}".format(usr_unique_mention_ct[user1]))           
    #print("usr_unique_mention_ct: {}".format(usr_unique_mention_ct))
    return usr_unique_mention_ct

In [43]:
def get_mention_fraction(mention_ct):
    print(type(mention_ct))
    count1_max = max(value[0] for value in mention_ct.values())
    count2_max = max(value[1] for value in mention_ct.values())
    mention_freq = []
    total_user1_num = len(mention_ct.keys())
    # print("total_user_num: {} count1_max: {} count2_max: {}".format(total_user1_num, count1_max, count2_max))
    for i in range(0, count2_max+1):
        freq = 0
        for counts in mention_ct.values():
            if counts[1] == i:
                freq += 1
        # print("num_seq: {} freq: {}".format(i, freq))
        mention_freq.insert(i, freq)
        
    mention_fraction = []
    for freq in mention_freq:
        mention_fraction.append(freq/total_user1_num)
    
    return mention_fraction
                

In [44]:
def graph_for_mention_frac(path, events):
    for event in events:
        file_name = os.path.join(path, "{}_mention_network.csv".format(event))
        mention_ct = count_unique_user2(file_name, event)
        mention_frac = get_mention_fraction(mention_ct)
        print(mention_frac)
        trace1 = Bar(
                x=['%s'%i for i in range(0, len(mention_frac)+1)],
                y=mention_frac,
                showlegend=False)
        layout=Layout(
                xaxis=XAxis(title='Num of unique user2'),
                yaxis=YAxis(title='Fraction of user1 who mentioned x number of user2'),
                title='Event {} User Mention Network Fraction'.format(event))

        data = Data([trace1])
        fig = Figure(data=data, layout=layout)
        plotly.offline.iplot(fig, filename="my plot")

graph_for_mention_frac(mention_path, [42, 44, 78])

<class 'dict'>
[0.7680525164113785, 0.18818380743982493, 0.0262582056892779, 0.006564551422319475, 0.006564551422319475, 0.0, 0.002188183807439825, 0.0, 0.002188183807439825]


<class 'dict'>
[0.745136186770428, 0.19747081712062256, 0.04134241245136187, 0.009727626459143969, 0.003404669260700389, 0.00048638132295719845, 0.0019455252918287938, 0.0, 0.00048638132295719845]


file: ./aggregated_mention_networks/78_mention_network.csv error: invalid literal for int() with base 10: 'https://pbs.twimg.com/profile_banners/123669878/1468044592'
<class 'dict'>
[0.6753163543168349, 0.2098350152170431, 0.06855678359762934, 0.0245074483421432, 0.011052378664103796, 0.005606279032516418, 0.002723049815793689, 0.0011212558065032837, 0.0004805382027871216, 0.00032035880185808104, 0.0, 0.00032035880185808104, 0.0, 0.00016017940092904052]
